In [1]:
from ultralytics import YOLO
import cv2 as cv
import numpy as np

In [ ]:
# 쓰레기 클래스 정의
trash_classes = {
    0: 'plastic',
    1: 'paper', 
    2: 'glass',
    3: 'metal',
    4: 'organic',
    5: 'cardboard'
}

# 클래스별 색상 정의
colors = {
    0: (255, 0, 0),    # plastic - 빨강
    1: (0, 255, 0),    # paper - 초록
    2: (0, 0, 255),    # glass - 파랑
    3: (255, 255, 0),  # metal - 노랑
    4: (255, 0, 255),  # organic - 자홍
    5: (0, 255, 255)   # cardboard - 청록
}

def draw_custom_boxes(frame, boxes, classes, confidences):
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        class_id = int(classes[i])
        confidence = confidences[i]
        
        # 바운딩 박스 그리기
        color = colors.get(class_id, (255, 255, 255))
        cv.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        # 라벨 텍스트
        label = f"{trash_classes.get(class_id, 'unknown')}: {confidence:.2f}"
        
        # 텍스트 배경
        text_size = cv.getTextSize(label, cv.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
        cv.rectangle(frame, (x1, y1 - text_size[1] - 10), 
                    (x1 + text_size[0], y1), color, -1)
        
        # 텍스트 그리기
        cv.putText(frame, label, (x1, y1 - 5), 
                  cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    
    return frame

def filter_trash_detections(results):
    filtered_boxes = []
    filtered_classes = []
    filtered_confidences = []
    
    if results[0].boxes is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()
        
        for i, class_id in enumerate(classes):
            if int(class_id) in trash_classes:
                filtered_boxes.append(boxes[i])
                filtered_classes.append(classes[i])
                filtered_confidences.append(confidences[i])
    
    return filtered_boxes, filtered_classes, filtered_confidences

cap = cv.VideoCapture(0)
model = YOLO("yolo11n.pt")

# 프레임 정보 표시용 변수
frame_count = 0
detection_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    frame_count += 1
    
    # YOLO 예측
    results = model.predict(frame, conf=0.5, verbose=False)
    
    # 쓰레기 객체만 필터링
    boxes, classes, confidences = filter_trash_detections(results)
    
    # 탐지된 객체가 있으면 바운딩 박스 그리기
    if boxes:
        detection_count += len(boxes)
        frame = draw_custom_boxes(frame, boxes, classes, confidences)
        
        # 탐지 정보 표시
        info_text = f"Detected: {len(boxes)} trash objects"
        cv.putText(frame, info_text, (10, 30), 
                  cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # 프레임 정보 표시
    status_text = f"Frame: {frame_count} | Total Detections: {detection_count}"
    cv.putText(frame, status_text, (10, frame.shape[0] - 10), 
              cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # 클래스 범례 표시
    legend_y = 60
    for class_id, class_name in trash_classes.items():
        color = colors[class_id]
        cv.rectangle(frame, (10, legend_y), (30, legend_y + 15), color, -1)
        cv.putText(frame, class_name, (35, legend_y + 12), 
                  cv.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        legend_y += 20
    
    cv.imshow("Trash Detection", frame)
    
    if cv.waitKey(1) == ord("q"):
        break

cap.release()
cv.destroyAllWindows()

print(f"총 {frame_count}개 프레임에서 {detection_count}개의 쓰레기 객체 탐지 완료")